#### calculate RMSE and SSIM

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import sys
import scipy.ndimage
import matplotlib.cm
import imageio
import skimage

In [2]:
sys.path.append('../../Preprocess/')
import CalcParaMaps
import BiasCorrection

In [34]:
frames = {}
cbfs = {}
cbvs = {}
mtts = {}
ttps = {}
N0 = 100000
imgNorm = 0.15
iTests = np.arange(55,70)

In [35]:
# optimal parameters
params = {
    100000: {'gauss': '2', 'tips': '0.075_3', 'tv': '7.5e-4_1e-4', 'n2n': '25', 'sup': '12.5'},
    200000: {'gauss': '2', 'tips': '0.05_2', 'tv': '5e-4_1e-4', 'n2n': '12.5', 'sup': '12.5'},
    1e+06: {'gauss': '1', 'tips': '0.025_2', 'tv': '1e-4_1e-4', 'n2n': '12.5', 'sup': '0'}
}

In [36]:
# read references
with np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/data/paras_tikh_0.3.npz') as f:
    mask = f['mask'][iTests, ...]
    cbfFac = f['cbfFac']
    cbvFac = f['cbvFac']
aif = np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/data/aif0.npy') / 1000 / imgNorm
ref = (np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/data/imgs_-1.npy')[iTests, ...] - 1) / imgNorm
img = (np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/data/imgs_%g.npy'%N0)[iTests, ...] - 1) / imgNorm

In [37]:
# vessel mask from img
maskVessels = np.where(np.max(img, -1) > 0.1 / imgNorm, 1, 0)
maskVessels *= mask
for i in range(maskVessels.shape[0]):
    maskVessels[i,...] = scipy.ndimage.morphology.binary_dilation(maskVessels[i,...])
mask *= (1-maskVessels)

img *= mask[...,np.newaxis]
ref *= mask[...,np.newaxis]
ctp = img - (img[...,[0]] + img[...,[1]]) / 2
refCtp = ref - ref[...,[0]]

In [38]:
def GetResults(ctp, aif, mask, cbfFac, cbvFac):
    cbf, cbv, mtt = CalcParaMaps.CalcParaMaps(ctp, mask[...,np.newaxis], kappa = 1, rho = 1, aif = np.copy(aif), directCBV = False)
    cbf *= cbfFac
    cbv *= cbvFac
    mtt = mtt / cbfFac * cbvFac
    ttp = np.argmax(ctp, -1)
    
    return cbf * mask, cbv * mask, mtt * mask, ttp * mask

In [39]:
def RMSE(src, ref, mask, th = None):
    if th is not None:
        src[src < th[0]] = th[0]
        src[src > th[1]] = th[1]
    return np.sqrt(np.sum((src - ref)**2 * mask, (1,2)) / np.sum(mask, (1,2)))

def SSIM(src, ref, mask, th = None):
    if th is not None:
        src[src < th[0]] = th[0]
        src[src > th[1]] = th[1]
    ssims = []
    for iSlice in range(src.shape[0]):
        maskSlice = mask[iSlice, ...]
        srcSlice = src[iSlice, ...] * maskSlice
        refSlice = ref[iSlice, ...] * maskSlice
        _, s = skimage.measure.compare_ssim(srcSlice, refSlice, data_range = srcSlice.max() - srcSlice.min(), full=True)
        
        s = np.sum(s * maskSlice) / np.sum(maskSlice)
        ssims.append(s)
    
    return np.array(ssims)

def SSIMTime(src, ref, mask):
    ssims = []
    for iSlice in range(src.shape[0]):
        ssimSlice = []
        for iFrame in range(src.shape[-1]):
            maskSlice = mask[iSlice, ...]
            srcSlice = src[iSlice, ..., iFrame] * maskSlice
            refSlice = ref[iSlice, ..., iFrame] * maskSlice
            _, s = skimage.measure.compare_ssim(srcSlice, refSlice, data_range = srcSlice.max() - srcSlice.min() + 1e-6, full=True)

            s = np.sum(s * maskSlice) / np.sum(maskSlice)
            ssimSlice.append(s)
        
        ssims.append(ssimSlice)
    
    return np.array(ssims)

In [40]:
# paramaps for reference
cbf0, cbv0, mtt0, ttp0 = GetResults(refCtp, aif, mask, cbfFac, cbvFac)

In [41]:
# paramaps for unfiltered
gauss = np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/gaussian/Gaussian_std_0_N0_%g.npy'%N0)
cbf, cbv, mtt, ttp = GetResults(gauss, aif, mask, cbfFac, cbvFac)

name = 'raw'
frames[name] = [RMSE(gauss, refCtp, mask[...,np.newaxis]), SSIMTime(gauss, refCtp, mask)]
cbfs[name] = [RMSE(cbf, cbf0, mask), SSIM(cbf, cbf0, mask)]
cbvs[name] = [RMSE(cbv, cbv0, mask), SSIM(cbv, cbv0, mask)]
mtts[name] = [RMSE(mtt, mtt0, mask, [0, 10]), SSIM(mtt, mtt0, mask, [0, 10])]
ttps[name] = [RMSE(ttp, ttp0, mask), SSIM(ttp, ttp0, mask)]

In [42]:
# paramaps for gaussian
name = 'gauss'
gauss = np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/gaussian/Gaussian_std_%s_N0_%g.npy'%(params[N0][name], N0))
cbf, cbv, mtt, ttp = GetResults(gauss, aif, mask, cbfFac, cbvFac)

frames[name] = [RMSE(gauss, refCtp, mask[...,np.newaxis]), SSIMTime(gauss, refCtp, mask)]
cbfs[name] = [RMSE(cbf, cbf0, mask), SSIM(cbf, cbf0, mask)]
cbvs[name] = [RMSE(cbv, cbv0, mask), SSIM(cbv, cbv0, mask)]
mtts[name] = [RMSE(mtt, mtt0, mask, [0, 10]), SSIM(mtt, mtt0, mask, [0, 10])]
ttps[name] = [RMSE(ttp, ttp0, mask), SSIM(ttp, ttp0, mask)]

In [43]:
# paramaps for TIPS
name = 'tips'
tips = np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/tips/TIPS_sigma_%s_N0_%g.npz.npy'%(params[N0][name], N0))
cbf, cbv, mtt, ttp = GetResults(tips, aif, mask, cbfFac, cbvFac)

frames[name] = [RMSE(tips, refCtp, mask[...,np.newaxis]), SSIMTime(tips, refCtp, mask)]
cbfs[name] = [RMSE(cbf, cbf0, mask), SSIM(cbf, cbf0, mask)]
cbvs[name] = [RMSE(cbv, cbv0, mask), SSIM(cbv, cbv0, mask)]
mtts[name] = [RMSE(mtt, mtt0, mask, [0, 10]), SSIM(mtt, mtt0, mask, [0, 10])]
ttps[name] = [RMSE(ttp, ttp0, mask), SSIM(ttp, ttp0, mask)]

In [44]:
# paramaps for TV
# this is a little different
name = 'tv'
with np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/tv/TV_beta_%s_N0_%g.npz'%(params[N0][name], N0)) as f:
    cbf = f['cbf'] * mask
    cbv = f['cbv'] * mask * cbvFac
    mtt = f['mtt'] * mask * cbvFac
    ttp = f['ttp'] * mask

# frames[name] = [RMSE(tips, refCtp, mask[...,np.newaxis]), SSIMTime(tips, refCtp, mask)]
cbfs[name] = [RMSE(cbf, cbf0, mask), SSIM(cbf, cbf0, mask)]
cbvs[name] = [RMSE(cbv, cbv0, mask), SSIM(cbv, cbv0, mask)]
mtts[name] = [RMSE(mtt, mtt0, mask, [0, 10]), SSIM(mtt, mtt0, mask, [0, 10])]
ttps[name] = [RMSE(ttp, ttp0, mask), SSIM(ttp, ttp0, mask)]

In [45]:
# paramaps for Noise2Noise
name = 'n2n'
n2n = np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/beta_%s_N0_%g/tmp/iodines.npy'%(params[N0][name], N0))[iTests, ...].transpose(0, 2, 3, 1) * 0.025 / imgNorm
# n2n = BiasCorrection.BiasCorrection(n2n, ctp, mask[...,np.newaxis])
cbf, cbv, mtt, ttp = GetResults(n2n, aif, mask, cbfFac, cbvFac)

frames[name] = [RMSE(n2n, refCtp, mask[...,np.newaxis]), SSIMTime(n2n, refCtp, mask)]
cbfs[name] = [RMSE(cbf, cbf0, mask), SSIM(cbf, cbf0, mask)]
cbvs[name] = [RMSE(cbv, cbv0, mask), SSIM(cbv, cbv0, mask)]
mtts[name] = [RMSE(mtt, mtt0, mask, [0, 10]), SSIM(mtt, mtt0, mask, [0, 10])]
ttps[name] = [RMSE(ttp, ttp0, mask), SSIM(ttp, ttp0, mask)]

In [46]:
# paramaps for supervised
name = 'sup'
sup = np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/supervised_beta_%s_N0_%g/tmp/iodines.npy'%(params[N0][name], N0))[iTests, ...].transpose(0, 2, 3, 1) * 0.025 / imgNorm
# sup = BiasCorrection.BiasCorrection(sup, ctp, mask[...,np.newaxis])
cbf, cbv, mtt, ttp = GetResults(sup, aif, mask, cbfFac, cbvFac)

frames[name] = [RMSE(sup, refCtp, mask[...,np.newaxis]), SSIMTime(sup, refCtp, mask)]
cbfs[name] = [RMSE(cbf, cbf0, mask), SSIM(cbf, cbf0, mask)]
cbvs[name] = [RMSE(cbv, cbv0, mask), SSIM(cbv, cbv0, mask)]
mtts[name] = [RMSE(mtt, mtt0, mask, [0, 10]), SSIM(mtt, mtt0, mask, [0, 10])]
ttps[name] = [RMSE(ttp, ttp0, mask), SSIM(ttp, ttp0, mask)]

In [47]:
# save all the data
outDir = '/home/dwu/trainData/Noise2Noise/train/ctp/results/simul/quantification'
if not os.path.exists(outDir):
    os.makedirs(outDir)
np.savez(os.path.join(outDir, 'rmse_ssim_%g'%N0), frames = frames, cbfs = cbfs, cbvs = cbvs, mtts = mtts, ttps = ttps)

In [48]:
for k in cbfs:
    print (k, np.mean(cbfs[k], 1))

raw [20.24007109  0.38977341]
gauss [5.56558539 0.75418261]
tips [4.78185902 0.78576604]
tv [9.49342699 0.61082187]
n2n [4.29339435 0.83941159]
sup [3.77868921 0.86162251]
